In [9]:
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [4]:
dataset = datasets.CIFAR10(root="data", train=True, download=True, transform=transforms.ToTensor())
loader = DataLoader(dataset, batch_size=1000, shuffle=False)

In [10]:
mean = 0.
std = 0.
total_images = 0

for images, _ in loader:
    batch_size = images.size(0)
    channels = images.size(1)

    images = images.view(batch_size, channels, -1) # (B, C, H * W)

    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)
    total_images += batch_size

mean /= total_images
std /= total_images

print(mean, std)


tensor([0.4914, 0.4822, 0.4465]) tensor([0.2023, 0.1994, 0.2010])
